# What if we imagine F# as more than it is?

In [1]:
#load ".paket/load/FluentValidation.fsx"
#load ".paket/load/FSharpPlus.fsx"
#load ".paket/load/System.ComponentModel.Annotations.fsx"
open FSharpPlus
open FSharpPlus.Operators
open FSharpPlus.Data
open System

What are the limitations that an ocaml or f# developer has to deal with?
 

In [16]:
let v = [(1,"alpha");(1,"beta");(1,"gamma");(2,"alfa");(3,"oe")]
        |> List.groupBy fst
        |> List.map snd
v

[[(1, "alpha"); (1, "beta"); (1, "gamma")]; [(2, "alfa")]; [(3, "oe")]]

In [17]:
let v = [(1,"alpha");(1,"beta");(1,"gamma");(2,"alfa");(3,"oe")]
        |> groupBy fst
        |> map snd
v

[[(1, "alpha"); (1, "beta"); (1, "gamma")]; [(2, "alfa")]; [(3, "oe")]]

## Composable validation errors

In order to glue together separate models and avoid Create methods that wrap validation and duplicate constructor parameter logic

In [2]:
type VError= | MustNotBeEmpty
             | MustBeAtLessThanChars of int
             | MustBeADate
             | MustBeOlderThan of int
             | MustBeWithingRange of decimal*decimal
module String=
    let nonEmpty (x:string) : Validation<VError list,string> = 
        if String.IsNullOrEmpty x 
        then Failure [MustNotBeEmpty]
        else Success x
    let mustBeLessThan (i:int) (x:string) : Validation<VError list,string> = 
        if isNull x || x.Length > i
        then Failure [MustBeAtLessThanChars i]
        else Success x
module Number=
    let mustBeWithin (from,to') (x)=
        if from<= x && x <= to'
        then Success x
        else Failure [MustBeWithingRange (from,to')]
module DateTime=
    let classicMovie year (d:DateTime)=
        if d.Year < year
        then Success d
        else Failure [MustBeOlderThan year]
    let date (d:DateTime)=
        if d.Date = d
        then Success d
        else Failure [MustBeADate]
type Genre=
    |Classic
    |PostClassic
    |Modern
    |PostModern
    |Contemporary
type Movie = {
    Id: int
    Title: String
    ReleaseDate: DateTime
    Description: String
    Price: decimal
    Genre: Genre
}
with static member Create(id,title,releaseDate,description,price,genre): Validation<VError list,Movie> =
        fun title releaseDate description price->{ Id=id;Title=title;ReleaseDate=releaseDate;Description=description;Price=price;Genre=genre }
        <!> String.nonEmpty title <* String.mustBeLessThan 100 title
        <*> DateTime.classicMovie 1960 releaseDate <* DateTime.date releaseDate
        <*> String.nonEmpty description <* String.mustBeLessThan 1000 description
        <*> Number.mustBeWithin (0.0m, 999.99m) price


In [3]:
Movie.Create(1,"Midsommar",DateTime(2019,6,24),"Midsommar is a 2019 folk horror film written...",1m,Classic) //Failure [MustBeOlderThan 1960]

Failure [MustBeOlderThan 1960]

In [4]:
Movie.Create(2,"Modern Times",DateTime(1936,2,5),"Modern Times is a 1936 American comedy film...",1m,Classic) // Success..

Success {Id = 2;
         Title = "Modern Times";
         ReleaseDate = 02/05/1936 00:00:00;
         Description = "Modern Times is a 1936 American comedy film...";
         Price = 1M;
         Genre = Classic;}

In [5]:
Movie.Create(3, String.Concat (seq{  1..110 }), DateTime(1950,1,1),"11",1m,Classic) //Failure [MustBeAtLessThanChars 100]

Failure [MustBeAtLessThanChars 100]

## Be aware of your abstractions 

[Suave using asp.net core](https://github.com/wallymathieu/FSharpPlus.AspNetCore/blob/master/src/FSharpPlus.AspNetCore.Suave/Library.fs)

Note how the WebPart defined

The below code uses OptionT, a strongly typed monad transformer. If you want to glue together separate parts without having to code your own combinations of monads.

In [6]:
type WebPart<'a> = 'a -> OptionT<Async<'a option>>

module WebPart=
  /// Entry-point for composing the applicative routes of the http application,
  /// by iterating the options, applying the context, arg, to the predicate
  /// from the list of options, until there's a match/a Some(x) which can be
  /// run.
  let choose (options : WebPart<'a> list) = fun x -> choice (List.map ((|>) x) options)
  let inline fail (_:'a) : OptionT<Async<'a option>> = async.Return None |> OptionT

## Generic lenses



In [7]:
open FSharpPlus.Lens

// From Mauricio Scheffer: https://gist.github.com/mausch/4260932
type Person = {
    Name: string
    DateOfBirth: DateTime
}
module Person=
    let inline _name f { Name = a; DateOfBirth = b } = f a <&> fun a' -> { Name = a'; DateOfBirth = b }
 type Book = {
    Title: string
    Author: Person
}
module Book =
    let inline _author f { Author = a; Title = b } = f a <&> fun a' -> { Author = a'; Title = b }
    let inline _authorName b = _author << Person._name <| b
let rayuela =
    { Book.Title = "Rayuela"
      Author = { Person.Name = "Julio Cortázar"
                 DateOfBirth = DateTime (1914, 8, 26) } }


In [10]:
// read book author name:
view Book._authorName rayuela

"Julio Cortázar"

In [9]:
//  you can also write the read operation as:
rayuela ^. Book._authorName

"Julio Cortázar"

## What use are lenses anyway

Main reason to use lenses is to deal with immutable data. If you use c# you probaly want to [generate code](http://assertfail.gewalli.se/2020/01/26/Immutable-classes-in-csharp.html) in some way or leverage f# in order to give you [simplified lenses](https://github.com/wallymathieu/with).